In [1]:
from openai import OpenAI
orgID = %env CHAT_GPT_ORGANIZATION_ID
api_key = %env CHAT_GPT_API_KEY

client = OpenAI(
    api_key = api_key,
    organization=orgID,
)

from googleapiclient.discovery import build
import json
import re
from isodate import parse_duration
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

api_key = %env YOUTUBE_SCAP_KEY
youtube = build('youtube', 'v3', developerKey=api_key)



In [2]:
channel_id = 'UCcefcZRL2oaA_uBNeo5UOWg' #ycombinator
video_id = 'UEngvxZ11sw'



def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return ''


tran = get_transcript(video_id)


justText = ''
textAndTimestamps = ''
for text in tran:
    textAndTimestamps += str(int(text['start'])) + '\n' + text['text'] + '\n'
    justText += text['text'] + '\n'



In [3]:
node_prefix = 21
node_start_sym = 'a'
topics = [{
	"start": 140,
	"finish": 277
},{
	"start": 291,
	"finish": 552 
},{
	"start": 563,
	"finish": 739 
},{
	"start": 743,
	"finish": 828 
},{
	"start": 848,
	"finish": 1014 
}
]




In [6]:
import os

startNum = node_prefix
startChar = node_start_sym
path = '/Users/shota/github/content-ge/startup-school/nodes'

def getTopic(topic, video_id):
    return {
        "name": "",
        "desctiption": "",
        "texts": ["text1.md"],
        "video_parts": [{
            "id": video_id,
            "start": topic["start"],
            "end": topic["finish"]
        }],
        "node_type": "lecture",
        "parent_id": "startup-school",
        "thumbnail": ""
    }



def make_folder(name):
    newNode = os.path.join(path, name)
    if not os.path.exists(newNode):
        os.makedirs(newNode)
    newFile = os.path.join(newNode, "text1.md")
    f = open(newFile, 'w')
    f.close()
    


for topic in topics:
    name = str(startNum) + startChar
    print('"' + name + '": ' + json.dumps(getTopic(topic, video_id), indent=4) + ',')
    make_folder(name)
    startChar = chr(ord(startChar) + 1)


"21a": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "7HDO1p3VdYg",
            "start": 140,
            "end": 277
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"21b": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "7HDO1p3VdYg",
            "start": 291,
            "end": 552
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"21c": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "7HDO1p3VdYg",
            "start": 563,
            "end": 739
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"21d": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
   

In [7]:
systemsText = """
You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.

You will be presented with the transcipt of a topic. Your goal is to produce:
1. Name of the topic
2. Very very small description of the topic
3. Rewrite this topic from your own perspective, user simple words, use markdown to highlight. Be the best 1-on-1 tutor
"""

def overlaps(a1, b1, a2, b2):
    int1 = pd.Interval(a1, b1)
    int2 = pd.Interval(a2, b2)
    return int1.overlaps(int2)

def getTranscriptForTopic(topic, transcript):
    start = topic['start']
    end = topic['finish']
    text = ''
    for snip in transcript:
        if overlaps(start, end, snip['start'], snip['start'] + snip['duration']):
            text+=snip['text']+' '
    return text


startNum = node_prefix
startChar = node_start_sym

for topic in topics:
    t = getTranscriptForTopic(topic, tran)
    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
              "role": "system",
                "content": systemsText
            },
            {
              "role": "user",
                "content": t
            }
        ],)
    print("====================================")
    name = str(startNum) + startChar
    newFile = os.path.join(os.path.join(path, name), "text1.md")
    f = open(newFile, 'w')
    print(resp.choices[0].message.content)
    f.write(resp.choices[0].message.content)
    f.close()
    startChar = chr(ord(startChar) + 1)
    print("====================================")

1. Topic Name: Observations on Leadership in Startups

2. Very Small Description: Understanding the diversity and authenticity of effective leadership styles in startups, and the importance of being true to oneself.

3. Rewritten Topic:

Let's talk about this very important thing, **Leadership**. Now when it comes to leadership, I have three things I've learned that I'd like to share with you. 

The first thing is, **there isn't just one 'cookie-cutter' image of what a good leader looks like**. Across all different types of people, there are great leaders. This was a bit of a surprise for me too. I used to think, there was a standard model or style of leadership, a specific way to talk or act to become a respected leader. But, as I came to work with diverse leaders, I realized that they're all very different. 

You have people who are quiet and introspective, and others who are outgoing and sociable. There are those drawn to technology and those attracted to storytelling. Some are cool

1. **Topic Name:** Building Trust as a Leader in a Startup.
2. **Small Description:** This topic discusses the importance of trust in leadership, with special emphasis on startups. It explains how trust, as a measurable metric, impacts the relationships between leaders and their employees, investors, and customers. It also explores the two aspects of building trust: the science, which relates to making correct empirical business decisions, and the art, which involves displaying empathy and good judgment.
3. **Topic Rewrite:**

Hello there, budding entrepreneurs! Today's lesson is about developing trust as a leader in **your future startup**. Now, why are we talking about trust? Well, trust is like a valuable gem that shows what kind of a leader you are. It's something that will define your relationship with everyone involved in your business. This includes your employees, your clients, your partners - you name it!

Think of trust like a score or grade for leaders. When people trust you